# Data Processing : Spark SQL, Dataframes, Datasets

## Prérequis

* Bonne compréhension du traitement des données à l'aide de Scala.
* Cycle de vie du traitement des données
   * Lecture de données à partir de fichiers
   * Traitement des données à l'aide d'API
   * Réécriture des données traitées dans des fichiers
* Nous pouvons également utiliser des bases de données comme sources et puits.

Nous aurons un aperçu du cycle de vie du traitement des données.
* Lire les données à partir d'un fichier.
* Prévisualisez le schéma et les données pour comprendre les caractéristiques des données.
* Obtenez un aperçu des API Data Frame ainsi que des fonctions utilisées pour traiter les données.
* Vérifiez s'il y a des doublons dans les données.
* Obtenez un aperçu sur comment écrire des données se trouvant dans des dataframes dans des fichiers en utilisant des formats de fichier compressès tels que Parquet.
* Nous allons approfondir les API Data Frame pour traiter les données dans les notebooks suivants.

## Rappel sur Spark Context
* `SparkSession`est un classe contenue dans le package `org.apache.spark.sql`
* Lorsque Spark Application est soumise à l'aide de `spark-submit` ou` spark-shell` ou `pyspark`, un service Web appelé Spark Context est démarré.

* Voici un exemple qui montre comment démarrer Spark Shell en locale.
```
spark-shell \
    --master "local[*]"
```
* Voici un exemple qui montre comment démarrer Spark Shell en mode multinodes dans un cluster.
```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0
```
* **Assurez-vous de bien comprendre l'environnement et utilisez la commande appropriée pour lancer Spark Shell.**
* Spark Context maintient le contexte de tous les jobs qui sont soumis jusqu'à ce qu'il soit *'killé'*.
* `SparkSession` n'est rien d'autre qu'un wrapper au-dessus de Spark Context.
* Nous devons d'abord créer un objet SparkSession avec n'importe quel nom. Mais généralement, nous utilisons «spark». Une fois créé, plusieurs API seront exposées, y compris `read`.
* Nous devons au moins définir le nom de l'application et également spécifier le mode d'exécution dans lequel Spark Context doit s'exécuter lors de la création de l'objet `SparkSession`.
* Nous pouvons utiliser `appName` pour spécifier le nom de l'application et` master` pour spécifier le mode d'exécution.
* Vous trouverez ci-dessous l'exemple d'extrait de code qui démarrera l'objet Spark Session pour nous.

In [1]:
import org.apache.spark.sql.SparkSession

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.21.226:4043
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1653138395084)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession


In [2]:
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    appName("Data Processing").
    master("local[8]").
    getOrCreate

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1238dda3


la tab permet de lister la lister des fonctions disponibles

![](images/tab.png)

In [3]:
spark.sparkContext.getConf.getAll.foreach(println)

(spark.driver.port,44527)
(spark.rdd.compress,True)
(spark.repl.class.outputDir,/tmp/tmpzpdgv5ei)
(spark.serializer.objectStreamReset,100)
(spark.master,local[*])
(spark.submit.pyFiles,)
(spark.executor.id,driver)
(spark.submit.deployMode,client)
(spark.repl.class.uri,spark://192.168.1.47:44527/classes)
(spark.app.name,spylon-kernel)
(spark.driver.host,192.168.1.47)
(spark.app.id,local-1639821967495)
(spark.ui.showConsoleProgress,true)


## Les APIs de lecture dans Spark

Aperçu des API de lecture Spark pour lire des fichiers de différents formats.

* `spark` a un tas d'API pour lire les données de fichiers de différents formats.
* Toutes les API sont exposées sous `spark.read`
     * `text` - pour lire les données d'une seule colonne à partir de fichiers texte ainsi que pour lire chacun des fichiers texte entiers comme un enregistrement.
     * `csv`- pour lire les fichiers texte avec des délimiteurs. La valeur par défaut est une virgule, mais nous pouvons également utiliser d'autres délimiteurs.
     * `json` - pour lire les données des fichiers JSON
     * `orc` - pour lire les données des fichiers ORC
     * `parquet` - pour lire les données des fichiers Parquet.
     * Nous pouvons également lire les données d'autres formats de fichiers en branchant et en utilisant `spark.read.format`
     * Nous pouvons également passer des options basées sur les formats de fichiers. 
     * `inferSchema` - pour déduire les types de données des colonnes en fonction des données.
     * `header` - pour utiliser l'en-tête pour obtenir les noms de colonne dans le cas de fichiers texte.
     * `schema` - pour spécifier explicitement le schéma.
     
* Voyons un exemple sur la façon de lire des données délimitées à partir de fichiers texte.


In [3]:
!tail datasets/retail_db/orders/part-00000

68874,2014-07-03 00:00:00.0,1601,COMPLETE
68875,2014-07-04 00:00:00.0,10637,ON_HOLD
68876,2014-07-06 00:00:00.0,4124,COMPLETE
68877,2014-07-07 00:00:00.0,9692,ON_HOLD
68878,2014-07-08 00:00:00.0,6753,COMPLETE
68879,2014-07-09 00:00:00.0,778,COMPLETE
68880,2014-07-13 00:00:00.0,1117,COMPLETE
68881,2014-07-19 00:00:00.0,2518,PENDING_PAYMENT
68882,2014-07-22 00:00:00.0,10000,ON_HOLD
68883,2014-07-23 00:00:00.0,5533,COMPLETE



In [4]:
// spark.read.csv
val orders = spark.
    read.
    schema("""order_id INT, order_date TIMESTAMP,
              order_customer_id INT, order_status STRING
           """).
    csv("datasets/retail_db/orders")

orders: org.apache.spark.sql.DataFrame = [order_id: int, order_date: timestamp ... 2 more fields]


In [6]:
// spark.read.csv avec option

val orders = spark.
    read.
    schema("""order_id INT, order_date TIMESTAMP,
              order_customer_id INT, order_status STRING
           """).
    option("sep", ",").
    csv("datasets/retail_db/orders")

orders: org.apache.spark.sql.DataFrame = [order_id: int, order_date: timestamp ... 2 more fields]


In [12]:
// spark.read.format

val orders = spark.
    read.
    schema("""order_id INT, order_date TIMESTAMP,
              order_customer_id INT, order_status STRING
           """).
    option("sep", ",").
    format("csv").
    load("datasets/retail_db/orders/")

orders: org.apache.spark.sql.DataFrame = [order_id: int, order_date: timestamp ... 2 more fields]


In [11]:
val orders = spark.
    read.
    option("inferSchema", "True").
    option("sep", ",").
    format("csv").
    load("datasets/retail_db/orders")

orders: org.apache.spark.sql.DataFrame = [_c0: int, _c1: string ... 2 more fields]


* Lecture de données JSON à partir de fichiers texte. Nous pouvons déduire un schéma à partir des données car chaque objet JSON contient à la fois le nom et la valeur de la colonne.
* Exemple pour JSON

```
{ "order_id": 1, "order_date": "2013-07-25 00:00:00.0", "order_customer_id": 12345, "order_status": "COMPLETE" }
```

In [5]:
// spark.read.json

val orders = spark.
    read.
    schema("""order_id INT, order_date TIMESTAMP,
              order_customer_id INT, order_status STRING
           """).
    json("datasets/retail_db_json/orders")

orders: org.apache.spark.sql.DataFrame = [order_id: int, order_date: timestamp ... 2 more fields]


In [14]:
// spark.read.format

val orders = spark.
    read.
    option("inferSchema", "false").
    schema("""order_id INT, order_date TIMESTAMP,
              order_customer_id INT, order_status STRING
           """).
    format("json").
    load("datasets/retail_db_json/orders")

orders: org.apache.spark.sql.DataFrame = [order_id: int, order_date: timestamp ... 2 more fields]


## Schéma et Données

Voici les API qui peuvent être utilisées pour prévisualiser le schéma et les données.

* `printSchema` peut être utilisé pour obtenir les détails du schéma.
* `show` peut être utilisé pour prévisualiser les données. Il affichera généralement les 20 premiers enregistrements où la sortie est tronquée.
* `describe` peut être utilisé pour extraire des statistiques de nos données.
* Nous pouvons transmettre le nombre d'enregistrements et définir truncate sur false lors de la prévisualisation des données.

In [6]:
val orders = spark.
    read.
    schema("""order_id INT, 
              order_date STRING, 
              order_customer_id INT, 
              order_status STRING
           """
          ).
    csv("datasets/retail_db/orders")

orders: org.apache.spark.sql.DataFrame = [order_id: int, order_date: string ... 2 more fields]


In [16]:
orders

res1: org.apache.spark.sql.DataFrame = [order_id: int, order_date: string ... 2 more fields]


In [7]:
// Print Schema
orders.printSchema

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)



In [18]:
// Describe
orders.describe().show(false)

+-------+------------------+---------------------+-----------------+---------------+
|summary|order_id          |order_date           |order_customer_id|order_status   |
+-------+------------------+---------------------+-----------------+---------------+
|count  |68883             |68883                |68883            |68883          |
|mean   |34442.0           |null                 |6216.571098819738|null           |
|stddev |19884.953633337947|null                 |3586.205241263963|null           |
|min    |1                 |2013-07-25 00:00:00.0|1                |CANCELED       |
|max    |68883             |2014-07-24 00:00:00.0|12435            |SUSPECTED_FRAUD|
+-------+------------------+---------------------+-----------------+---------------+



In [8]:
// Preview Data - Default
orders.show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|

In [22]:
// Preview Data - 10, with truncate false
orders.show(10, truncate=false)

+--------+---------------------+-----------------+---------------+
|order_id|order_date           |order_customer_id|order_status   |
+--------+---------------------+-----------------+---------------+
|1       |2013-07-25 00:00:00.0|11599            |CLOSED         |
|2       |2013-07-25 00:00:00.0|256              |PENDING_PAYMENT|
|3       |2013-07-25 00:00:00.0|12111            |COMPLETE       |
|4       |2013-07-25 00:00:00.0|8827             |CLOSED         |
|5       |2013-07-25 00:00:00.0|11318            |COMPLETE       |
|6       |2013-07-25 00:00:00.0|7130             |COMPLETE       |
|7       |2013-07-25 00:00:00.0|4530             |COMPLETE       |
|8       |2013-07-25 00:00:00.0|2911             |PROCESSING     |
|9       |2013-07-25 00:00:00.0|5657             |PENDING_PAYMENT|
|10      |2013-07-25 00:00:00.0|5648             |PENDING_PAYMENT|
+--------+---------------------+-----------------+---------------+
only showing top 10 rows



## L'API Dataframe

- Les transformations au niveau de la ligne ou la projection de données peuvent être effectuées à l'aide de `select, selectExpr, withColumn, drop` sur un Dataframe.
- Nous pouvons appliquer les fonctions de `org.apache.spark.sql.functions` sur les colonnes en utilisant `select` et `withColumn`
- Le filtrage est généralement effectué à l'aide d'un filtre ou sur un Dataframe.
- Nous pouvons passer une condition pour filtrer ou en utilisant le style SQL ou le style d'un langage de programmation.
- Les agrégations globales peuvent être effectuées directement sur un Dataframe
- Les agrégations sont généralement effectuées à l'aide de groupBy, puis les fonctions sont aggrées à l'aide de `agg`.
- Le données dans un Data Frame peuven etre triées ou ordonnées en utilisant `sort` ou `orderBy`



Etudions comment projeter les données en utilisant différentes options telles que `select`, `selectExpr`, `withColumn`, `drop`.

In [10]:
val employees = List((1, "Scott", "Tiger", 1000.0, "united states"),
                     (2, "Henry", "Ford", 1250.0, "India"),
                     (3, "Nick", "Junior", 750.0, "united KINGDOM"),
                     (4, "Bill", "Gomes", 1500.0, "AUSTRALIA")
                    )

employees: List[(Int, String, String, Double, String)] = List((1,Scott,Tiger,1000.0,united states), (2,Henry,Ford,1250.0,India), (3,Nick,Junior,750.0,united KINGDOM), (4,Bill,Gomes,1500.0,AUSTRALIA))


In [11]:
// Creation du Dataframe employees

val employeesDF = employees.
    toDF("employee_id", 
         "first_name", 
         "last_name", 
         "salary", 
         "nationality"
        )


employeesDF: org.apache.spark.sql.DataFrame = [employee_id: int, first_name: string ... 3 more fields]


In [12]:
employeesDF.printSchema

root
 |-- employee_id: integer (nullable = false)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- salary: double (nullable = false)
 |-- nationality: string (nullable = true)



In [13]:
employeesDF.show()

+-----------+----------+---------+------+--------------+
|employee_id|first_name|last_name|salary|   nationality|
+-----------+----------+---------+------+--------------+
|          1|     Scott|    Tiger|1000.0| united states|
|          2|     Henry|     Ford|1250.0|         India|
|          3|      Nick|   Junior| 750.0|united KINGDOM|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|
+-----------+----------+---------+------+--------------+



In [14]:
// projection de first_name et de last_name
employeesDF.select("first_name", "last_name").show

+----------+---------+
|first_name|last_name|
+----------+---------+
|     Scott|    Tiger|
|     Henry|     Ford|
|      Nick|   Junior|
|      Bill|    Gomes|
+----------+---------+



In [15]:
// Projecter tous les données souf Nationality ()
employeesDF.drop("nationality").show

+-----------+----------+---------+------+
|employee_id|first_name|last_name|salary|
+-----------+----------+---------+------+
|          1|     Scott|    Tiger|1000.0|
|          2|     Henry|     Ford|1250.0|
|          3|      Nick|   Junior| 750.0|
|          4|      Bill|    Gomes|1500.0|
+-----------+----------+---------+------+



In [16]:
employeesDF.show()

+-----------+----------+---------+------+--------------+
|employee_id|first_name|last_name|salary|   nationality|
+-----------+----------+---------+------+--------------+
|          1|     Scott|    Tiger|1000.0| united states|
|          2|     Henry|     Ford|1250.0|         India|
|          3|      Nick|   Junior| 750.0|united KINGDOM|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|
+-----------+----------+---------+------+--------------+



In [17]:
val empl= employeesDF.drop("nationality")
empl.show()

+-----------+----------+---------+------+
|employee_id|first_name|last_name|salary|
+-----------+----------+---------+------+
|          1|     Scott|    Tiger|1000.0|
|          2|     Henry|     Ford|1250.0|
|          3|      Nick|   Junior| 750.0|
|          4|      Bill|    Gomes|1500.0|
+-----------+----------+---------+------+



empl: org.apache.spark.sql.DataFrame = [employee_id: int, first_name: string ... 2 more fields]


In [30]:
val employeesDF2 = employeesDF.drop("nationality")

employeesDF2: org.apache.spark.sql.DataFrame = [employee_id: int, first_name: string ... 2 more fields]


In [31]:
employeesDF2.show

+-----------+----------+---------+------+
|employee_id|first_name|last_name|salary|
+-----------+----------+---------+------+
|          1|     Scott|    Tiger|1000.0|
|          2|     Henry|     Ford|1250.0|
|          3|      Nick|   Junior| 750.0|
|          4|      Bill|    Gomes|1500.0|
+-----------+----------+---------+------+



Plus de details dans le next step...

## Les fonctions

Dans Spark, differentes fonctions existent pour analyser et traiter les données.

- Les fonctions sont souvent appliquées sur les valeurs de colonnes.
- Les fonctions Spark qui permettent de traiter les colonnes d'un Dataframe sont disponibles dans `org.apache.spark.sql.functions`. Celles-ci sont généralement utilisés dans `select` ou `withColumn` au-dessus du bloc Dataframe
-  Il y a environ 300 fonctions prédéfinies disponibles dans Spark. Les plus importantes sont regroupées sous forme de fonctions de manipulation de chaînes de caractères, de manipulation de date, de fonctions numériques et  de fonctions d'agrégation :
    
    - Fonctions de manipulation de chaînes caractères :
        * Concaténation de chaînes - `concat`
        * longueur d'une chaine - `length`
        * Suppression d'espace debut et fin - `trim, rtrim, ltrim`
        * Padding - lpad, rpad
        * Extraction de chaînes - `split, substring`
    - Fonctions de manipulation de date
        * calcul de date - `date_add, date_sub, dateiff, add_months`
        * Extraction de date - `dayofmonth, month, year, date_format`
        * Obtenir la période de début - `trunc, date_trunc`
    - Fonctions numériques - `abs, greatest`
    - Fonctions d'agrégation - `sum, min, max`

In [20]:
import org.apache.spark.sql.functions.col

import org.apache.spark.sql.functions.col


In [19]:
employeesDF.select("employee_id").show()

+-----------+
|employee_id|
+-----------+
|          1|
|          2|
|          3|
|          4|
+-----------+



In [21]:
col("employee_id")

res10: org.apache.spark.sql.Column = employee_id


In [22]:
employeesDF.select(col("employee_id")).show

+-----------+
|employee_id|
+-----------+
|          1|
|          2|
|          3|
|          4|
+-----------+



In [24]:
// En utilisant col et lit
import org.apache.spark.sql.functions.{col, lit, concat}
employeesDF.
    select(col("employee_id"),
           concat(col("first_name"), lit(" "), col("last_name")).alias("full_name"),
           col("salary"),
           col("nationality")
          ).show

+-----------+-----------+------+--------------+
|employee_id|  full_name|salary|   nationality|
+-----------+-----------+------+--------------+
|          1|Scott Tiger|1000.0| united states|
|          2| Henry Ford|1250.0|         India|
|          3|Nick Junior| 750.0|united KINGDOM|
|          4| Bill Gomes|1500.0|     AUSTRALIA|
+-----------+-----------+------+--------------+



import org.apache.spark.sql.functions.{col, lit, concat}


In [25]:
employeesDF.
    withColumn("full_name", 
               concat(col("first_name"), lit(", "), col("last_name"))).
    drop("first_name", "last_name").show

+-----------+------+--------------+------------+
|employee_id|salary|   nationality|   full_name|
+-----------+------+--------------+------------+
|          1|1000.0| united states|Scott, Tiger|
|          2|1250.0|         India| Henry, Ford|
|          3| 750.0|united KINGDOM|Nick, Junior|
|          4|1500.0|     AUSTRALIA| Bill, Gomes|
+-----------+------+--------------+------------+



In [34]:
val newEmployes = employeesDF.
    withColumn("full_name", 
               concat(col("first_name"), lit(", "), col("last_name"))).
    drop("first_name", "last_name")

newEmployes: org.apache.spark.sql.DataFrame = [employee_id: int, salary: double ... 2 more fields]


In [35]:
newEmployes.show

+-----------+------+--------------+------------+
|employee_id|salary|   nationality|   full_name|
+-----------+------+--------------+------------+
|          1|1000.0| united states|Scott, Tiger|
|          2|1250.0|         India| Henry, Ford|
|          3| 750.0|united KINGDOM|Nick, Junior|
|          4|1500.0|     AUSTRALIA| Bill, Gomes|
+-----------+------+--------------+------------+



In [36]:
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    appName("Data Processing").
    master("yarn").
    getOrCreate

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4ae4b618


In [37]:
import spark.implicits._

import spark.implicits._


In [ ]:
$"colname" equvalent col("colname")

In [26]:
// En utilisant $ et lit
employeesDF.
    withColumn("full_name", 
               concat($"first_name", lit(", "), $"last_name")).
    drop("first_name", "last_name").
    show

+-----------+------+--------------+------------+
|employee_id|salary|   nationality|   full_name|
+-----------+------+--------------+------------+
|          1|1000.0| united states|Scott, Tiger|
|          2|1250.0|         India| Henry, Ford|
|          3| 750.0|united KINGDOM|Nick, Junior|
|          4|1500.0|     AUSTRALIA| Bill, Gomes|
+-----------+------+--------------+------------+



In [39]:
spark.sql("SHOW functions").show(300, false)

+---------------------------+
|function                   |
+---------------------------+
|!                          |
|!=                         |
|%                          |
|&                          |
|*                          |
|+                          |
|-                          |
|/                          |
|<                          |
|<=                         |
|<=>                        |
|<>                         |
|=                          |
|==                         |
|>                          |
|>=                         |
|^                          |
|abs                        |
|acos                       |
|acosh                      |
|add_months                 |
|aggregate                  |
|and                        |
|any                        |
|approx_count_distinct      |
|approx_percentile          |
|array                      |
|array_contains             |
|array_distinct             |
|array_except               |
|array_int

In [27]:
spark.sql("DESCRIBE FUNCTION concat").show(false)

+------------------------------------------------------------------------------------------+
|function_desc                                                                             |
+------------------------------------------------------------------------------------------+
|Function: concat                                                                          |
|Class: org.apache.spark.sql.catalyst.expressions.Concat                                   |
|Usage: concat(col1, col2, ..., colN) - Returns the concatenation of col1, col2, ..., colN.|
+------------------------------------------------------------------------------------------+



In [40]:
// En utilisant Style SQL 
employeesDF.
    selectExpr("employee_id",
               "concat(first_name, ' ', last_name) AS full_name",
               "salary", 
               "nationality"
              ).show

+-----------+-----------+------+--------------+
|employee_id|  full_name|salary|   nationality|
+-----------+-----------+------+--------------+
|          1|Scott Tiger|1000.0| united states|
|          2| Henry Ford|1250.0|         India|
|          3|Nick Junior| 750.0|united KINGDOM|
|          4| Bill Gomes|1500.0|     AUSTRALIA|
+-----------+-----------+------+--------------+



## Vue d'ensemble sur les APIs de Lecture/écriture de Spark

Tous les APIs sont exposés par `spark.read`.
- **txt** - pour écrire des données à une seule colonne dans des fichiers texte.
- **csv** - pour écrire dans des fichiers texte avec des délimiteurs. La valeur par défaut est une virgule, mais nous pouvons également utiliser d'autres délimiteurs.
- **json** - pour écrire des données dans des fichiers JSON
- **orc** - pour écrire des données dans des fichiers ORC
- **parquet** - pour écrire des données dans des fichiers Parquet.

 Nous pouvons également écrire des données dans d'autres formats de fichiers en branchant et en utilisant `write.format `, par exemple **avro**.  
 Nous pouvons utiliser des options basées sur le type dans lequel nous écrivons le Dataframe.
 - **compression** - Codec de compression (gzip, snappy, etc.)
 - **sep** - pour spécifier des délimiteurs lors de l'écriture dans des fichiers texte en utilisant csv
 
 Nous pouvons écraser les répertoires ou les ajouter aux répertoires existants en utilisant `mode`.
 Créez une copie des données de commande au format de fichier parquet sans compression. Si le dossier existe déjà, écrasez-le. Emplacement cible: `/user/[YOUR_USER_NAME]/retail_db/orders`  
 Par défaut, le nombre de fichiers dans le répertoire de sortie est égal au nombre de tâches utilisées pour traiter les données à la dernière étape. Cependant, nous pourrions vouloir contrôler le nombre de fichiers afin de ne pas nous heurter à un trop grand nombre de petits fichiers.  
Nous pouvons contrôler le nombre de fichiers en utilisant `coalesce`. Il doit être appelé au-dessus du Dataframe avant d'appeler `write`.


In [41]:
val orders = spark.
    read.
    schema("""order_id INT, 
              order_date STRING, 
              order_customer_id INT, 
              order_status STRING
           """
          ).
    csv("datasets/retail_db/orders")

orders: org.apache.spark.sql.DataFrame = [order_id: int, order_date: string ... 2 more fields]


In [42]:
spark.conf.get("spark.sql.parquet.compression.codec")

res19: String = snappy


In [43]:
// En utilisant write.parquet
orders.
    write.
    mode("overwrite").
    option("compression", "none").
    parquet("datasets/training/retail_db/orders")

In [44]:
// En utilisant write.format("parquet")
orders.
    coalesce(1).
    write.
    mode("overwrite").
    option("compression", "none").
    format("parquet").
    save("datasets/training/retail_db/orders")

* Vous pouvez excuter cette commande dans un cluster hadoop pour lister les chuncks crées 

In [ ]:
import sys.process._

"hdfs dfs -ls  datasets/training/retail_db/orders" !

In [45]:
spark.read.parquet("datasets/training/retail_db/orders").printSchema

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)



In [46]:
spark.read.parquet("datasets/training/retail_db/orders").show

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|